In [1]:
#dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import gmaps.datasets
from scipy.stats import linregress
from scipy import stats
import os

In [2]:
#load assesment file
Hills_Values = os.path.join ("../../GB/Resources/HillsboroughAsmt.csv")
#create dataframe
Hills_Values_df = pd.read_csv(Hills_Values, dtype={'PHY_ZIPCD':'str','TOT_LVG_AREA':'str','ACT_YR_BLT':'str'})
Hills_Values_df.head()

crime_path = os.path.join ("../Resources/combined_crime_data.csv")
crime_data = pd.read_csv(crime_path)

In [3]:
#pick necessary columns
Home_Values_df = Hills_Values_df [['SEQ_NO','PARCEL_ID','ASMNT_YR','JV','NBRHD_CD','CENSUS_BK','PHY_ADDR1','PHY_CITY','PHY_ZIPCD','TOT_LVG_AREA','ACT_YR_BLT']]
Home_Values_df.head()

,SEQ_NO,PARCEL_ID,ASMNT_YR,JV,NBRHD_CD,CENSUS_BK,PHY_ADDR1,PHY_CITY,PHY_ZIPCD,TOT_LVG_AREA,ACT_YR_BLT
0,1,153323ZZZ000000000200A,2020,4628604,22700200,120579900000,EGMONT KEY,ST PETERSBURG,33715,NaN,NaN
1,2,153323ZZZ000000000201A,2020,100,22700200,120579900000,NaN,NaN,NaN,NaN,NaN
2,3,153323ZZZ000000000400A,2020,208044,22700200,120579900000,NaN,ST PETERSBURG,33715,NaN,NaN
3,4,172701001000000000001U,2020,147699,21100700,120570115091,19507 PINE TREE RD,ODESSA,33556,NaN,NaN
4,5,172701001000000000010U,2020,140829,21100700,120570115092,19931 ANGEL LN,ODESSA,33556,NaN,NaN


In [4]:
#remove any null values 
home_value1_df =Home_Values_df.dropna()
home_value1_df.head()

,SEQ_NO,PARCEL_ID,ASMNT_YR,JV,NBRHD_CD,CENSUS_BK,PHY_ADDR1,PHY_CITY,PHY_ZIPCD,TOT_LVG_AREA,ACT_YR_BLT
5,6,172701001000000000011U,2020,462229,21100700,120570115092,19859 ANGEL LN,ODESSA,33556,3972,1996
6,7,172701001000000000012U,2020,496434,21100700,120570115091,7320 W COUNTY LINE RD,ODESSA,33556,4643,2003
7,8,172701001000000000013U,2020,358083,21100700,120570115091,7212 W COUNTY LINE RD,ODESSA,33556,3269,1999
8,9,172701001000000000014U,2020,819704,21100700,120570115091,7222 W COUNTY LINE RD,ODESSA,33556,8203,2008
9,10,172701001000000000015U,2020,325923,21100700,120570115091,7326 W COUNTY LINE RD,ODESSA,33556,2608,2003


In [5]:
#remame columns
renamed_df = home_value1_df.rename(columns={"JV": "PropertyValue",
                                            "PHY_ADDR1": "PropAddress",
                                            "ASMNT_YR": "AssmntYr",
                                            "PHY_CITY":"City",
                                            "PHY_ZIPCD":"ZipCode",
                                             "TOT_LVG_AREA":"BldgSqFt",
                                           "ACT_YR_BLT": "YearBuilt"})
renamed_df.head()

,SEQ_NO,PARCEL_ID,AssmntYr,PropertyValue,NBRHD_CD,CENSUS_BK,PropAddress,City,ZipCode,BldgSqFt,YearBuilt
5,6,172701001000000000011U,2020,462229,21100700,120570115092,19859 ANGEL LN,ODESSA,33556,3972,1996
6,7,172701001000000000012U,2020,496434,21100700,120570115091,7320 W COUNTY LINE RD,ODESSA,33556,4643,2003
7,8,172701001000000000013U,2020,358083,21100700,120570115091,7212 W COUNTY LINE RD,ODESSA,33556,3269,1999
8,9,172701001000000000014U,2020,819704,21100700,120570115091,7222 W COUNTY LINE RD,ODESSA,33556,8203,2008
9,10,172701001000000000015U,2020,325923,21100700,120570115091,7326 W COUNTY LINE RD,ODESSA,33556,2608,2003


In [6]:
#create data frame for specific zip code


Zip13_df = renamed_df.loc[(renamed_df["ZipCode"]=="33613")]
Zip56_df = renamed_df.loc[(renamed_df["ZipCode"]=="33556")]

# Zip13_df.head()
# Zip56_df.head()

In [7]:
# create dataframe with properties less than 2500 sqft
#the theory here is that most Fl homes are between 1000 and 2500sqft, this will help eliminate outliers that skew the average prop valueHills2020_df = Zip_df.loc[(Zip_df["BldgSqFt"]<"2500")
Hills13_df = Zip13_df.loc[(Zip13_df["BldgSqFt"] < "2500")]
Hills56_df = Zip56_df.loc[(Zip56_df["BldgSqFt"] < "2500")]
# Hills13_df.head()
# Hills56_df.head()

In [20]:
# Drop outlier indices

# Calculate the average property value by ZIP


home_average13_df = pd.DataFrame(Hills13_df["PropertyValue"]).mean()
home_average56_df = pd.DataFrame(Hills56_df["PropertyValue"]).mean()

# home_average13_df.head()
# home_average56_df.head()

PropertyValue    373204.791192
dtype: float64

In [12]:
crime_data.head()

,Unnamed: 0,Unnamed: 0.1,Reported Date,Case Number,Crime Type,Crime Type Extension,Address,zip,year
0,0,0,1/1/2016,2016-281,DUI,DUI (DRIVING UNDER INFLUENCE),FLORIDA AV N / WINDWOOD OAKS DR,33613,2016
1,1,6,1/1/2016,2016-60,BATTERY,BATTERY-SIMPLE,13600 BLOCK GRAGSTON CR,33613,2016
2,2,7,1/2/2016,2016-2990,BURGLARY,THEFT FROM A VEHICLE,4000 BLOCK DREAM OAK PL,33613,2016
3,3,9,1/2/2016,2016-3345,THEFT,THEFT FROM A BUILDING,13700 BLOCK 42ND ST N,33613,2016
4,4,10,1/2/2016,2016-3552,BATTERY,BATTERY-SIMPLE,14200 BLOCK NEBRASKA AV N,33613,2016


In [21]:
# Add average property value to crime dataframe
for i, row in home_average13_df.iterrows():
    crime_data.loc[crime_data['zip'] == row['ZipCode'], ["Avg Prop Value"] == row['PropertyValue']

crime_data.head()

SyntaxError: invalid syntax (<ipython-input-21-7b14590a3133>, line 5)

In [22]:
crime_data = pd.DataFrame(crime_data,columns=["Reported Date", "Case Number", "Crime Type", "Crime Type Extension",
                                              "year", "zip"])
crime_data.head()

,Reported Date,Case Number,Crime Type,Crime Type Extension,year,zip
0,1/1/2016,2016-281,DUI,DUI (DRIVING UNDER INFLUENCE),2016,33613
1,1/1/2016,2016-60,BATTERY,BATTERY-SIMPLE,2016,33613
2,1/2/2016,2016-2990,BURGLARY,THEFT FROM A VEHICLE,2016,33613
3,1/2/2016,2016-3345,THEFT,THEFT FROM A BUILDING,2016,33613
4,1/2/2016,2016-3552,BATTERY,BATTERY-SIMPLE,2016,33613


In [23]:
index_crime_19 = crime_data[(crime_data['year'] != 2019)].index 
crime_data_19 = crime_data.drop(index_crime_19, inplace = False)

crime_data_19.head()

,Reported Date,Case Number,Crime Type,Crime Type Extension,year,zip
6687,1/1/2019,2019-1374,BATTERY,BATTERY-SIMPLE,2019,33613
6688,1/1/2019,2019-1374,DRUGS,DRUGS/NARCOTICS,2019,33613
6689,1/1/2019,2019-1374,DRUGS,DRUG PARAPHERNALIA,2019,33613
6690,1/1/2019,2019-1202,BURGLARY,BURGLARY RESIDENCE / NO FORCE,2019,33613
6691,1/1/2019,2019-1342,BURGLARY,BURGLARY RESIDENCE / NO FORCE,2019,33613


In [24]:
#Create scatter plot w/ regression line for school grade vs. prop value

# Set x and y values for scatter plot
x_values = crime_data_19['Avg Prop Value']
y_values = crime_data_19['Crime Type']
# y_values = crime_data_19['Reported Date']
# y_values = crime_data_19['Case Number']
# y_values = crime_data_19['year']
# y_values = crime_data_19['zip']

# Perform a linear regression on school grade vs. average property value
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

# Get regression values
regress_values = x_values * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))

# Create scatter plot and
# set plot specs
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Average Property Value in School ZIP (by $)')
plt.ylabel("School Grade (% of total possible points)")
plt.ylim(0, 100)
plt.xticks(rotation=45)
plt.annotate(line_eq,(5,-40),fontsize=15,color="red")
plt.grid(which='major', axis='both')
plt.title("School Grade vs. Average Property Value")

# Print r value
print(f"The r- is: {rvalue**2}")

# Display plot
plt.show()

KeyError: 'Avg Prop Value'